# VacationPy
---

### Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies, setup, API keys
import hvplot.pandas
hvplot.extension('bokeh')
import pandas as pd
import requests
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tamanrasset,22.7850,5.5228,282.10,37,0,2.06,DZ,1731480255
1,1,bethel,41.3712,-73.4140,275.49,61,0,0.89,US,1731480257
2,2,okhotsk,59.3833,143.3000,258.63,70,11,7.16,RU,1731480258
3,3,basco,20.4487,121.9702,299.98,89,65,6.50,PH,1731480259
4,4,fort st. john,56.2499,-120.8529,266.09,93,75,2.06,CA,1731480260


In [ ]:
# Convert max temp column to Fahrenheit - probably should have done this in WeatherPy
city_data_df['Max Temp (F)'] = round( ((city_data_df['Max Temp'] - 273.15) * 9/5 + 32), 1 )

# Replace temp column w/ new one
city_data_df = city_data_df[['City_ID', 'City', 'Lat', 'Lng', 'Max Temp (F)', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date']]
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tamanrasset,22.7850,5.5228,48.1,37,0,2.06,DZ,1731480255
1,1,bethel,41.3712,-73.4140,36.2,61,0,0.89,US,1731480257
2,2,okhotsk,59.3833,143.3000,5.9,70,11,7.16,RU,1731480258
3,3,basco,20.4487,121.9702,80.3,89,65,6.50,PH,1731480259
4,4,fort st. john,56.2499,-120.8529,19.3,93,75,2.06,CA,1731480260


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City'
)

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [70]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df.loc[
    # temp
    (city_data_df['Max Temp (F)'] < 80) & (city_data_df['Max Temp (F)'] > 50)
    # cloudiness
    & (city_data_df['Cloudiness'] < 80)
    # wind
    & (city_data_df['Wind Speed'] < 10)
    # humidity
    & (city_data_df['Humidity'] < 80)
    ].dropna()

# Display sample data
filtered_city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
14,14,port douglas,-16.4833,145.4667,79.5,69,56,2.42,AU,1731480272
15,15,blackmans bay,-43.0167,147.3167,64.5,58,18,5.39,AU,1731480273
25,25,maues,-3.3836,-57.7186,77.9,69,13,0.91,BR,1731480284
32,32,los frentones,-26.4077,-61.4137,58.4,50,1,1.65,AR,1731480292
36,36,waitangi,-43.9535,-176.5597,52.8,78,2,0.89,NZ,1731480296
37,37,hasaki,35.7333,140.8333,62.4,51,16,7.82,JP,1731480297
40,40,puerto baquerizo moreno,-0.9000,-89.6000,69.2,76,41,4.52,EC,1731480006
43,43,santa cruz,-17.8000,-63.1667,63.9,77,75,5.14,BO,1731479989
46,46,lihue,21.9789,-159.3672,74.8,74,75,7.72,US,1731480307
47,47,bredasdorp,-34.5322,20.0403,66.4,53,47,5.00,ZA,1731480308


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = [''] * len(hotel_df)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,callao,PE,-12.0667,-77.1500,77,
14,port douglas,AU,-16.4833,145.4667,69,
15,blackmans bay,AU,-43.0167,147.3167,58,
18,hawaiian paradise park,US,19.5933,-154.9731,89,
20,puerto ayora,EC,-0.7393,-90.3518,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Import json
import json

In [94]:
# Choose hotels
categories = ['accommodation.hotel']

# Set params dictionary
params = {
    'categories':categories,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},10000'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # to make sure it's working: print(json.dumps(name_address))

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
callao - nearest hotel: Casa Ronald
port douglas - nearest hotel: Mantra Aqueous Port Douglas
blackmans bay - nearest hotel: Villa Howden
hawaiian paradise park - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
adamstown - nearest hotel: No hotel found
maues - nearest hotel: No hotel found
sanming - nearest hotel: 三明客家宾馆
uturoa - nearest hotel: Hawaiki Nui hotel
los frentones - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
hasaki - nearest hotel: 大新旅館
camacupa - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
crane - nearest hotel: No hotel found
santa cruz - nearest hotel: Aviador Hotel Boutique
lihue - nearest hotel: Kauai Palms
bredasdorp - nearest hotel: Victoria Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
arawa - nearest hotel: Bovotel
lagunas - nearest hotel: Hospedaje Eco
wailua homesteads - nearest hotel: Hilton Garden Inn Kau

,City,Country,Lat,Lng,Humidity,Hotel Name
8,callao,PE,-12.0667,-77.1500,77,Casa Ronald
14,port douglas,AU,-16.4833,145.4667,69,Mantra Aqueous Port Douglas
15,blackmans bay,AU,-43.0167,147.3167,58,Villa Howden
18,hawaiian paradise park,US,19.5933,-154.9731,89,No hotel found
20,puerto ayora,EC,-0.7393,-90.3518,87,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
572,uribia,CO,11.7139,-72.2660,87,No hotel found
576,alesund,NO,62.4723,6.1549,72,Brosundet
577,alupka,UA,44.4167,34.0500,80,Сон у моря
579,galeana,MX,24.8333,-100.0667,67,Hotel Jardin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [97]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
    )
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)